In [1]:
# matrix with all the gene expression values for a particular condition 

genes, gene lengths, 
observed- gene expressionn values
for neg bino reg model - need parameter for mean expression level and dispersion parameter 
mean expression level can be stratified by catogery

In [1]:
import os

# These MUST come before importing pymc or pytensor
os.environ["PYTENSOR_FLAGS"] = "optimizer_excluding=constant_folding,mode=FAST_RUN,linker=py"
os.environ["PYTENSOR_WARN__C_COMPILE"] = "False"

import pytensor
pytensor.config.cxx = ""

# Now import pymc
import pymc as pm
import arviz as az
print("Linker:", pytensor.config.linker)

import numpy as np
import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Linker: py


In [2]:
import scanpy as sc
import matplotlib.pyplot  as plt

In [3]:
import pymc as pm
print("PyMC version:", pm.__version__)


PyMC version: 5.12.0


In [4]:
adata = sc.read("merged_dataset.h5ad")

In [5]:
# Print a summary of the dataset
print(adata)

AnnData object with n_obs × n_vars = 186224 × 39241
    obs: 'AvgSpotLen', 'Bases', 'Bytes', 'original_index'


In [6]:
# adata_sub = adata[:500, :100].copy()
adata_sub = adata[:1000, :500].copy()

# Extract raw count matrix (dense format)
# Convert to dense array if needed
if not isinstance(adata_sub.X, np.ndarray):
    counts = adata_sub.X.toarray()
else:
    counts = adata_sub.X
# Transpose to get (n_genes, n_cells)
observed_counts = counts.T.astype(int)
n_genes, n_conditions = observed_counts.shape  # "conditions" are cells

In [7]:
# Simulate realistic sequencing depth if NaN
if 'Bases' in adata_sub.obs.columns:
    # Generate simulated values for missing entries (e.g., log-normal)
    missing_mask = adata_sub.obs['Bases'].isna()
    simulated_bases = np.random.lognormal(mean=10, sigma=0.5, size=missing_mask.sum())
    adata_sub.obs.loc[missing_mask, 'Bases'] = simulated_bases

    # Convert to numeric and drop any remaining invalids just in case
    adata_sub.obs['Bases'] = pd.to_numeric(adata_sub.obs['Bases'], errors='coerce')
else:
    # Add the column entirely if it doesn't exist
    adata_sub.obs['Bases'] = np.random.lognormal(mean=10, sigma=0.5, size=adata_sub.n_obs)

In [8]:
# Simulate gene lengths and sequencing depths if not available
gene_lengths = np.random.uniform(1000, 3000, size=n_genes)
seq_depth = np.median(adata_sub.obs['Bases']) if 'Bases' in adata_sub.obs else 1e4

In [9]:
gene_lengths = np.array(gene_lengths)
seq_depth = float(seq_depth) 

In [10]:
ref_seq_depth = 1e5
ref_gene_length = 2000.0

In [11]:
seq_depth

30630.969549844973

In [ ]:
with pm.Model() as model_vi:
    # ---- Mini-batched Inputs ----
    # gene_indices aligns each observation to a gene index (0..n_genes-1)
    # condition_indices aligns each observation to a condition index (0..n_conditions-1)
    gene_idx = np.repeat(np.arange(n_genes), n_conditions)
    cond_idx = np.tile(np.arange(n_conditions), n_genes)
    obs_flat = observed_counts.flatten()
    gene_lengths_flat = gene_lengths[gene_idx]

    minibatch_obs = pm.Minibatch(obs_flat, batch_size=128)
    minibatch_gene_idx = pm.Minibatch(gene_idx, batch_size=128)
    minibatch_cond_idx = pm.Minibatch(cond_idx, batch_size=128)
    minibatch_gene_lengths = pm.Minibatch(gene_lengths_flat, batch_size=128)

    # Constants
    seq_depth_shared = pm.Data("seq_depth", np.array([seq_depth]))

    # ---- Non-centered Parameterization ----
    mu_gene = pm.Normal("mu_gene", mu=2.0, sigma=1.0)
    sigma_gene = pm.HalfNormal("sigma_gene", sigma=1.0)
    z_gene = pm.Normal("z_gene", mu=0, sigma=1, shape=n_genes)
    gene_log_expression = pm.Deterministic("gene_log_expression", mu_gene + sigma_gene * z_gene)

    mu_cond = pm.Normal("mu_cond", mu=0.0, sigma=1.0)
    sigma_cond = pm.HalfNormal("sigma_cond", sigma=1.0)
    z_cond = pm.Normal("z_cond", mu=0, sigma=1, shape=n_conditions)
    condition_intercepts_log = pm.Deterministic("condition_intercepts_log", mu_cond + sigma_cond * z_cond)

    # Dispersion
    alpha = pm.HalfCauchy("alpha", beta=2)

    # ---- Compute minibatch mu ----
    log_mu = gene_log_expression[minibatch_gene_idx] + condition_intercepts_log[minibatch_cond_idx]
    log_mu += pm.math.log(seq_depth_shared[0] / ref_seq_depth)
    log_mu += pm.math.log(minibatch_gene_lengths / ref_gene_length)
    mu = pm.math.exp(log_mu)

    # ---- Minibatch Likelihood ----
    pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=minibatch_obs)

    # ---- Fit with VI ----
    approx = pm.fit(
        n=300_000,
        method='advi',
        callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=0.01)],
        progressbar=True
    )

    trace_vi = approx.sample(draws=1000)


/scratch/hr2547/myenv/lib64/python3.9/site-packages/pymc/data.py:434: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


In [ ]:
az.summary(trace_vi)

In [ ]:
# --------------------------
# ELBO Plot for VI
# --------------------------
plt.plot(approx.hist)
plt.title("ELBO Loss Curve (VI)")
plt.xlabel("Iteration")
plt.ylabel("ELBO")
plt.tight_layout()
plt.show()